## Import Library:

In [1]:
import os # untuk berinteraksi dengan sistem operasi, seperti untuk mengakses file dan direktori
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split # untuk membagi dataset menjadi subset training dan testing
import torch # Library utama PyTorch untuk komputasi tensor dan deep learning
from torch.utils.data import Dataset, DataLoader # Kelas untuk memanipulasi dataset dan membuat batch data untuk training
from torchvision import transforms # Modul untuk melakukan transformasi pada gambar seperti augmentasi data
import torch.nn as nn # Modul yang berisi berbagai komponen neural network seperti lapisan (layers), fungsi aktivasi, dll
import torch.optim as optim # Modul untuk algoritma optimasi, seperti SGD, Adam, dll
# from torchvision.models import resnet50 # Model pre-trained ResNet-50 yang bisa digunakan untuk tugas klasifikasi gambar
import torchvision.models as models
import time # Library untuk mengukur waktu eksekusi kode
from torch.autograd import Variable # Kelas yang membungkus tensor untuk melacak sejarah operasi dan menghitung gradient
from PIL import Image # Library untuk membuka, memanipulasi, dan menyimpan gambar
from sklearn.metrics import classification_report, confusion_matrix # Fungsi untuk menghitung dan menampilkan laporan klasifikasi dan confusion matrix
import cv2 # Library OpenCV untuk manipulasi gambar dan video
from torchvision.io import read_image

## Validation:

In [2]:
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

In [3]:
model.load_state_dict(torch.load('/kaggle/input/celeba-gender-classification-best-model/pytorch/facerecognition/2/model_best_ResNet-18 Epoch 25 Weights BCE- Bella on Kaggle/model_best_ResNet-18 Epoch 25 Weights BCE- Bella on Kaggle.pth')['model'])
if torch.cuda.is_available():
  model = model.to("cuda")

In [4]:
val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [5]:
val_images_list = os.listdir('/kaggle/input/celeba-face-recognition/validation_images/validation_images')
val_images_list.sort()

In [6]:
val_images_list

['female_1.jpeg',
 'female_2.jpeg',
 'female_3.jpeg',
 'female_4.jpeg',
 'female_5.jpeg',
 'male_1.jpg',
 'male_2.jpeg',
 'male_3.jpeg',
 'male_4.jpeg',
 'male_5.jpeg']

In [7]:
inf_time_start = time.time()
val_pred = torch.empty((0)).cuda()
for i in val_images_list:
    image_path = os.path.join('/kaggle/input/celeba-face-recognition/validation_images/validation_images', i)
    image = Image.open(image_path).convert('RGB')
    inputs = val_transforms(image)
    inputs = Variable(inputs.cuda())
    inputs = inputs.unsqueeze(0)
    outputs = model(inputs)
      #print(outputs)
    preds = (torch.sigmoid(outputs).squeeze() > 0.5).float()
    val_pred = torch.cat((val_pred, preds.reshape(1)), -1)
print('Prediction Time for Each Image is {} ms'.format(1000 * (time.time() - inf_time_start) / len(val_images_list)))

Prediction Time for Each Image is 115.64722061157227 ms


In [ ]:
# inf_time_start = time.time()
# val_pred = torch.empty((0)).cuda()
# for i in val_images_list:
#     image_path = os.path.join('/kaggle/input/celeba-face-recognition/validation_images/validation_images', i)
#     image = Image.open(image_path).convert('RGB')
#     inputs = val_transforms(image)
#     inputs = Variable(inputs.cuda())
#     inputs = inputs.unsqueeze(0)
#     outputs = model(inputs)
#     preds = (torch.sigmoid(outputs).squeeze() > 0.5).float()
#     val_pred = torch.cat((val_pred, preds.reshape(1)), -1)
# print('Prediction Time for Each Image is {} s'.format((time.time() - inf_time_start) / len(val_images_list)))

In [8]:
val_pred

tensor([0., 1., 0., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')